In [1]:
!pip install langchain_ollama

- Platform : Ollama
- Model: llama3.2:3b
- Framework: langchain

In [2]:
from langchain_core.tools import Tool
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import Field
from langgraph.prebuilt import tools_condition
from langchain_ollama import ChatOllama

In [3]:
llm = ChatOllama(model="llama3.2:3b", temperature=0.7)

def search_tool(query: str) -> str:
    """Tool to give respecive role features and responsibilities."""
    return f"Searching: {query}"

tools = [
    Tool(
        llm=llm,
        name="Search",
        func=search_tool,
        description="Tool to give respective role features and responsibilities."
    )
]

In [4]:
task = "I want to develop, test and deploy a web and Artificial Intelligence based application for our retail Grocery store where my customers can browse, search an item, scan any item using barcode, add to cart, online payment, online-delivery, tracking of order and online feedback. You have to do the project in a Scrum-agile model with sprints. " + "You have to give me the estimates from every phase seperately. You take the conversations like this customer, project manager, requirements, Developers, Quality Assurance (QA) Engineers, Business Analysts(BAs), DevOps Engineers, Architects, Scrum Master, Security Engineers, and Technical Writers. All the responses should be not more than 200 words"

In [5]:
customer_prompt = PromptTemplate(
    template="""
    You are a retail store manager. You know every corner of your retail store.
    You can transform your complex concepts into requirements for an IT company's Requirement Engineer.
    
    Initial requirement:
    I want to develop, test, and deploy a web and AI-based application for our retail grocery store where my customers can:
    - Browse items
    - Search for items
    - Scan items using barcode
    - Add items to cart
    - Make online payments
    - Arrange online delivery
    - Track orders
    - Provide online feedback
    
    Provide these requirements to the project manager.
    """
)

scrum_master = PromptTemplate(
    template= """ 
        You are a scrum master overseeing this project. Your role is to plan sprints and check all the hits and misses of the sprint. You are performing this project in the scrum-agile model. 

        Your role is to deal with and answer all these questions:
        - You have to tell which features are to be completed in which sprint taking real-time scenarios.
        - No of sprints, releases and tasks with proper days with story points.
        - Acts as a facilitator for the Scrum Team and make a stand-up call everyday.
        - Serves the Product Owner, Development Team, and the Organization.
        - Ensures the Scrum process is followed correctly.
        - plan and explain the estimates in the duration in hours or days for each sprint.
        - Encourages continuous improvement and innovation.
        - Ensures engineering practices and quality standards are maintained.
        - At the end you are responsible to provide sprint wise features in a tabular form.
        
    """
)


product_owner = PromptTemplate(
    template= """ 
        You are a prodcut owner overseeing this project. 
        
        Your role is to deal with and answer all these questions:
        - No of sprints, releases and tasks with proper days with story points.
        - Acts as the primary point of contact between stakeholders and the development team for each sprint.
        - Owns and prioritizes the product backlog to ensure it reflects business value for each sprint.
        - According to the scrum master, You have to tell which features are to be completed in which sprint taking real-time scenarios for each sprint.
        - Works closely with the Scrum team to clarify requirements and provide feedback for each sprint.
        - Orders the product backlog based on business goals, customer needs, and team capacity.
        - Creates clear user stories and acceptance criteria for the development team. 
        - Reviews and approves completed work to ensure it meets quality standards.
        - Gathers input from stakeholders and balances their needs with project goals.
        - Participates in sprint planning, reviews, and retrospectives to guide the team for each sprint.
        - At the end you are responsible to provide sprint wise userstories, count of userstories in a tabular form.

    """
)

requirement_engineer_prompt = PromptTemplate(
    template="""
    You are a Requirement Engineer responsible for eliciting, analyzing, specifying, validating, and managing requirements.
    
    Received project plan:
    {project_plan}
    
    Now analyze and document detailed requirements:
    - According to the scrum master and product owner, You have to tell which features are to be completed in which sprint taking real-time scenarios.
    - Estimate total requirements pages (exact figure) for each sprint seperately with real scenario values.
    - Estimate productivity rate (pages per hour) (exact figure) for each sprint seperately with real scenario values.
    - List detailed feature for each sprint seperately with real scenario values.
    - At the end you are responsible to provide the sprint wise estimated total requirements pages and productivity rate (pages per hour) and donot give me fancy counts
    - Also give me proper total no of days to complete for this phase

    Provide detailed requirements to the system engineer.
    """
)

project_artitect = PromptTemplate(
    template="""
    You are a Project Artitect responsible for shaping the overall structure and design of the project.

    Received detailed requirements:
    {detailed_requirements}
    
    Now create the system design:
    - According to the scrum master and product owner, You have to tell which features are to be completed in which sprint taking real-time scenarios.
    - Define overall architecture (monolithic, microservices, serverless, etc.).
    - Choose appropriate tech stack (languages, frameworks, databases, cloud platforms).
    - Set coding standards, design principles, and architectural guidelines.
    - Lead design reviews and mentor developers for each sprint seperately with real scenario values.
    - Estimate total design document pages (exact figure) for each sprint seperately with real scenario values.
    - Estimate productivity rate (pages per hour) (exact figure) for each sprint seperately with real scenario values.
    - List design components (including database schema) for each sprint seperately with real scenario values.
    - list all the entity relationships, databases and tables names, foreign and primary keys for each sprint seperately with real scenario values for each sprint seperately with real scenario values.
    - Create high-level design documents and system blueprints 
    - Define modular structure, APIs, interfaces, and data flows for each sprint seperately with real scenario values.
    - Ensure design meets scalability, performance, and security requirements for each sprint seperately with real scenario values.
    - Address cross-cutting concerns (e.g., logging, error handling, observability)
    - At the end you are responsible to provide the sprint wise estimated total design in pages and productivity rate (pages per hour)
    - Also give me proper total no of days to complete for this phase

    Provide system design to the software engineer.
""" 
)


ui_ux_design__engineer_prompt = PromptTemplate(
    template="""
    You are a System Engineer responsible for designing, integrating, and managing complex systems.
    
    Received detailed requirements:
    {detailed_requirements}
    
    Now create the system design:
    - According to the scrum master and product owner, You have to tell which features are to be completed in which sprint taking real-time scenarios.
    - Estimate total design document pages (exact figure) for each sprint seperately with real scenario values.
    - Estimate productivity rate (pages per hour) (exact figure) for each sprint seperately with real scenario values.
    - List front end design components (generic and new components, user friendly UI) for each sprint seperately with real scenario values.
    - List the features in left navigation, their design, material design and other frontend related things.
    - At the end you are responsible to provide the sprint wise estimated total tasks and productivity rate (tasks per hour)
    - Also give me proper total no of days to complete for this phase

    Provide system design to the software engineer.
    """
)

business_analyst_prompt = PromptTemplate(
    template="""
    You are a business analyst and need to have a clearly understood, documented, and communicated to the technical team. In a software project, especially in Agile/Scrum, the BA acts as the bridge between stakeholders and developers.

    Now create the system design and do it for each sprint seperately with real scenario values:
    - According to the scrum master and product owner, You have to tell which features are to be completed in which sprint taking real-time scenarios.
    - Estimate total BA tasks count or document  pages (exact figure)
    - Estimate productivity rate (pages per hour) (exact figure)
    - create detailed User Stories / Use Cases
    - Acceptance Criteria
    - Process Flows / Diagrams
    - Evaluate the effectiveness of the solution post-implementation.
    - Suggest improvements or optimizations to the product and processes
    - At the end you are responsible to provide the sprint wise estimated total tasks and productivity rate (tasks per hour)
    - Also give me proper total no of days to complete for this phase

""")

cicd_emgineer_prompt = PromptTemplate(
    template="""
    A DevOps Engineer bridges the gap between development and operations teams by automating, streamlining, and integrating software development and IT operations. 
    
    - According to the scrum master and product owner, You have to tell which features are to be completed in which sprint taking real-time scenarios.
    - Design, implement, and maintain Continuous Integration (CI) and Continuous Deployment (CD) pipelines.
    - Ensure smooth, automated builds, tests, and deployments.
    - Integrate tools like Jenkins and GitLab CI/CD. Use tools like Terraform, or CloudFormation to define and manage environments.
    - Automate cloud resource provisioning (AWS, Azure, GCP).
    - Ensure consistency and versioning in infrastructure.
    - Estimate total DevOps tasks count / document (exact figure)
    - Estimate productivity rate (tasks per hour) (exact figure)
    - At the end you are responsible to provide the sprint wise estimated total tasks and productivity rate (tasks per hour)
    - Also give me proper total no of days to complete for this phase

    """
    )

security_engineer_prompt = PromptTemplate(
    template="""
        A Security Engineer is responsible for protecting software systems, networks, and data from threats and vulnerabilities.

        Answer the following for each sprint seperately with real scenario values:
        - According to the scrum master and product owner, You have to tell which features are to be completed in each sprint taking real-time scenarios.            
        - Identify and assess potential threats and vulnerabilities early in the design phase.
        - Perform risk analysis to prioritize and mitigate threats effectively.
        - Secure Software Development
        - Integrate security best practices into SDLC for each sprint seperately with real scenario values.
        - Review code for security flaws (manual or automated tools).
        - Promote the use of secure coding standards for each sprint seperately with real scenario values.
        - Collaborate with QA to include automated security testing in CI/CD pipelines.
        - Develop and test incident response plans.
        - Investigate security breaches and conduct root cause analysis.
        - Estimate total security breaches count / document (exact figure) for each sprint seperately with real scenario values.
        - At the end you are responsible to provide the sprint wise estimated total security breaches and productivity rate (security breaches per hour)
        - Also give me proper total no of days to complete for this phase

        """
    )

software_engineer_prompt = PromptTemplate(
    template="""
    You are a Software Engineer responsible for designing, developing, and maintaining software applications.
    
    Received system design:
    {system_design}
    
    Now develop the application for each sprint seperately with real scenario values.:
    - According to the scrum master and product owner, You have to tell which features are to be completed in which sprint taking real-time scenarios.
    - Estimate total SLOCs (Source Lines of Code) (exact figure) for each sprint seperately with real scenario values.
    - Estimate productivity rate (SLOCs per hour) (exact figure) for each sprint seperately with real scenario values.
    - Specify technology stack
    - List implementation details for each sprint seperately with real scenario values.
    - define which techology is better and why?
    - Do we need monolitic or microservice for this ?
    - At the end you are responsible to provide the sprint wise no of days and feature names.
    - At the end you are responsible to provide the sprint wise estimated total SLOC and productivity rate (SLOC per hour)
    - Also give me proper total no of days to complete for this phase

    Provide development details to the testing engineer.
    """
)

testing_engineer_prompt = PromptTemplate(
    template="""
    You are a Testing Engineer responsible for ensuring software quality through testing.
    
    Received development details:
    {development_details}
    
    Now create and execute test cases for each sprint seperately with real scenario values.:
    - According to the scrum master and product owner, You have to tell which features are to be completed in which sprint taking real-time scenarios.
    - Estimate total test cases (exact figure) for each sprint seperately with real scenario values.
    - Estimate productivity rate (test cases per hour) (exact figure) for each sprint seperately with real scenario values.
    - Create test plan for each sprint seperately with real scenario values.
    - per feature how many test cases (exact figure) for each sprint seperately with real scenario values.
    - At the end you are responsible to provide the sprint wise estimated total test cases and productivity rate (test cases per hour)
    - Also give me proper total no of days to complete for this phase 

    Provide testing details to the documentation engineer.
    """
)

documentation_engineer_prompt = PromptTemplate(
    template="""
    You are a Technical Writer responsible for creating and maintaining technical documentation.
    
    Received testing details:
    {testing_details}
    
    Now create project documentation for each sprint seperately with real scenario values.:
    - According to the scrum master and product owner, You have to tell which features are to be completed in which sprint taking real-time scenarios.
    - Estimate total documentation pages (exact figure) for each sprint seperately with real scenario values.
    - Estimate productivity rate (pages per hour) (exact figure) for each sprint seperately with real scenario values.
    - List documentation components for each sprint seperately with real scenario values.
    - The documentation engineer is responsible for documenting and display all the estimates (Phase, Engineer-Name, Amount of work and productivity of work) (exact figure)
    - At the end, draw a tabular column with No of sprints for each node, total estimates for each sprint, productivity rates for each sprint. 
    - Also give me proper total no of days to complete this project 
    Provide final documentation details.
    """
)

In [6]:
from langchain_core.messages import AIMessage, HumanMessage

# Define the state structure
class AgentState(TypedDict):
    messages: Annotated[Sequence[HumanMessage | AIMessage], lambda x, y: x + y]
    current_role: str

In [7]:
customer_chain = customer_prompt | llm | StrOutputParser()
# project_manager_chain = project_manager_prompt | llm | StrOutputParser()
requirement_engineer_chain = requirement_engineer_prompt | llm | StrOutputParser()
architect_chain = project_artitect | llm | StrOutputParser()
ui_ux_chain = ui_ux_design__engineer_prompt | llm | StrOutputParser()
business_analyst_chain = business_analyst_prompt | llm | StrOutputParser()
devops_chain = business_analyst_prompt | llm | StrOutputParser()
security_chain = security_engineer_prompt | llm | StrOutputParser()
software_engineer_chain = software_engineer_prompt | llm | StrOutputParser()
testing_engineer_chain = testing_engineer_prompt | llm | StrOutputParser()
documentation_chain = documentation_engineer_prompt | llm | StrOutputParser()
scrum_master_chain = scrum_master | llm | StrOutputParser()
product_owner_chain = product_owner | llm | StrOutputParser()

print(product_owner_chain)

first=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template=' \n        You are a prodcut owner overseeing this project. \n        \n        Your role is to deal with and answer all these questions:\n        - No of sprints, releases and tasks with proper days with story points.\n        - Acts as the primary point of contact between stakeholders and the development team for each sprint.\n        - Owns and prioritizes the product backlog to ensure it reflects business value for each sprint.\n        - According to the scrum master, You have to tell which features are to be completed in which sprint taking real-time scenarios for each sprint.\n        - Works closely with the Scrum team to clarify requirements and provide feedback for each sprint.\n        - Orders the product backlog based on business goals, customer needs, and team capacity.\n        - Creates clear user stories and acceptance criteria for the development team. \n        - Reviews and appr

In [8]:
from langchain_core.runnables import RunnableLambda

async def process_node(chain, input_key, state):
    last_message = state["messages"][-1].content
    inputs = {input_key: last_message} if input_key else {"input": last_message}
    
    full_response = ""
    async for chunk in chain.astream(inputs):
        full_response += chunk
        yield chunk
    
    state["messages"].append(HumanMessage(content=full_response))

In [9]:
async def customer_node_stream(state: AgentState):
    async for chunk in process_node(customer_chain, "input", state):
        yield chunk
    yield "\n\n--- Role Transition: Customer → Product Owner ---\n\n"

async def product_owner_node_stream(state: AgentState):
    async for chunk in process_node(product_owner_chain, None, state):
        yield chunk
    yield "\n\n--- Role Transition: Product Owner → Scrum Master ---\n\n"

async def scrum_master_node_stream(state: AgentState):
    async for chunk in process_node(scrum_master_chain, None, state):
        yield chunk
    yield "\n\n--- Role Transition: Scrum Master → Requirement Engineer ---\n\n"

async def requirement_engineer_node_stream(state: AgentState):
    async for chunk in process_node(requirement_engineer_chain, "project_plan", state):
        yield chunk
    yield "\n\n--- Role Transition: Requirement Engineer → Business Analyst ---\n\n"

async def business_analyst_node_stream(state: AgentState):
    async for chunk in process_node(business_analyst_chain, None, state):
        yield chunk
    yield "\n\n--- Role Transition: Business Analyst → Architect ---\n\n"

async def architect_node_stream(state: AgentState):
    async for chunk in process_node(architect_chain, "detailed_requirements", state):
        yield chunk
    yield "\n\n--- Role Transition: Architect → UI/UX Designer ---\n\n"

async def ui_ux_node_stream(state: AgentState):
    async for chunk in process_node(ui_ux_chain, "detailed_requirements", state):
        yield chunk
    yield "\n\n--- Role Transition: UI/UX Designer → Software Engineer ---\n\n"

async def software_engineer_node_stream(state: AgentState):
    async for chunk in process_node(software_engineer_chain, "system_design", state):
        yield chunk
    yield "\n\n--- Role Transition: Software Engineer → DevOps Engineer ---\n\n"

async def devops_node_stream(state: AgentState):
    async for chunk in process_node(devops_chain, None, state):
        yield chunk
    yield "\n\n--- Role Transition: DevOps Engineer → Security Engineer ---\n\n"

async def security_node_stream(state: AgentState):
    async for chunk in process_node(security_chain, None, state):
        yield chunk
    yield "\n\n--- Role Transition: Security Engineer → Testing Engineer ---\n\n"

async def testing_engineer_node_stream(state: AgentState):
    async for chunk in process_node(testing_engineer_chain, "development_details", state):
        yield chunk
    yield "\n\n--- Role Transition: Testing Engineer → Documentation Engineer ---\n\n"

async def documentation_node_stream(state: AgentState):
    async for chunk in process_node(documentation_chain, "testing_details", state):
        yield chunk
    yield "\n\n--- Role Transition: Documentation Engineer → Scrum Master ---\n\n"

# Main streaming workflow executor
async def run_streaming_workflow():
    state = AgentState(messages=[HumanMessage(content=task)], current_role="initiator")
    
    nodes = [
        ("customer", customer_node_stream),
        ("product_owner", product_owner_node_stream),
        ("scrum_master", scrum_master_node_stream),
        ("requirement_engineer", requirement_engineer_node_stream),
        ("business_analyst", business_analyst_node_stream),
        ("architect", architect_node_stream),
        ("ui_ux_designer", ui_ux_node_stream),
        ("software_engineer", software_engineer_node_stream),
        ("devops_engineer", devops_node_stream),
        ("security_engineer", security_node_stream),
        ("testing_engineer", testing_engineer_node_stream),
        ("documentation_engineer", documentation_node_stream),
    ]
    
    print("Starting Agile Scrum Workflow...\n")
    print("="*50 + "\n")
    
    for role_name, node_func in nodes:
        print(f"\n{role_name.upper()} OUTPUT:\n")
        print("-"*50)
        
        # Create a new generator for each node
        stream = node_func(state)
        
        # Process the stream
        try:
            async for chunk in stream:
                print(chunk, end="", flush=True)
        except Exception as e:
            print(f"\nError in {role_name}: {str(e)}")
            break


async def execute_workflow():
    await run_streaming_workflow()

# This works in notebook environments
import nest_asyncio
nest_asyncio.apply()

# Now run the workflow
await execute_workflow()

Starting Agile Scrum Workflow...



CUSTOMER OUTPUT:

--------------------------------------------------
**Project Requirements Document**

**Project Title:** Retail Grocery Store Web and AI-Based Application

**Overview:**
The objective of this project is to design, develop, test, and deploy a web-based application for our retail grocery store that enables customers to browse items, search for products, scan items using barcode, add items to cart, make online payments, arrange online delivery, track orders, and provide online feedback.

**Functional Requirements:**

1. **Item Browsing and Search**
	* Develop a user-friendly interface to display a list of available items.
	* Implement search functionality that allows customers to search for specific items by name, description, or keyword.
	* Display relevant search results with item images, descriptions, and prices.
2. **Barcode Scanning**
	* Integrate barcode scanning technology to allow customers to quickly identify and add items to 